<a href="https://colab.research.google.com/github/hwarang97/spam_classifier/blob/main/spam_classifier_ngram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [407]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score, accuracy_score
from nltk.stem import PorterStemmer
from collections import Counter
import re
import pandas as pd
import numpy as np

# Hyper Parameters

In [408]:
n = 3
SELECT = {
    'use_lower': True,
    'use_stemming': False,
    'use_stopwords': True,
    'use_numreplace': False
}
CV = 5
random_state = 42

결과를 얻기 편하도록 하이퍼 파라미터를 모아두었습니다.   
n은 n-gram에서 몇개의 토큰을 연결할지를 결정합니다.   
SELECT는 전처리에 사용할 방법들을 선택할 수 있습니다.   
(사용: True, 미사용: False)   
CV: Cross validation의 k 값
random_state: CV시 데이터를 나눌 때 시드설정

# Preprocessing

In [409]:
def preprocess_text(text, use_lower=False, use_stemming=False, use_stopwords=False, use_numreplace=False):
    if use_lower:
        text = lowercase_text(text)

    words = tokenize_text(text)

    if use_stemming:
        words = stem_words(words)

    if use_numreplace:
        words = replace_numbers(words)

    if use_stopwords:
        words = remove_stopwords(words)

    return words

def lowercase_text(text):
    return text.lower()

def stem_words(words):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

# Function to remove stopwords
def remove_stopwords(words):
    stopwords = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves',
                     'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
                     'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
                     'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with',
                     'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
                     'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some',
                     'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd',
                     'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
                     "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
                     "weren't", 'won', "won't", 'wouldn', "wouldn't"])

    return [word for word in words if word not in stopwords]

# Function to replace numbers with a special token "NUM"
def replace_numbers(words):
    return ['NUM' if word.isdigit() else word for word in words]

# Tokenize the text by words
def tokenize_text(text):
    return re.findall(r'\b\w+\b', text)

# Function to generate n-grams from a list of words
def generate_ngrams(words, n=2):
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = ' '.join(words[i:i + n])
        ngrams.append(ngram)
    return ngrams

- preprocess_text(text):    
전처리를 선택적으로 적용할 수 있는 함수. 하이퍼파라미터 SELECT에서 사용하기로 결정한 전처리만 적용되도록 하였습니다.

- lowercase_text(text):   
받아들인 text 값을 소문자로 변경합니다.

- stem_words(words):   
토튼화된 정보에서 어간을 추출하는 방법으로 포터 알고리즘을
사용하였습니다. 기본적으로 소문자화 기능을 가지고 있기에, stem_words를 사용할때는 lowercase_text 함수를 사용하지 않았습니다.

- replace_numbers(words):   
토큰화된 정보 중 숫자에 해당되는 것들은 'NUM'으로 변경하였습니다.   
NUM으로 변경한 이유는 아래와 같습니다.   
    - 1. 대다수의 빈도수는 1이다.   
    빈도수가 1밖에 안되는것은 다시 반복될 가능성이 낮습니다. 따라서 일반적으로 판단하기에 부적절한 데이터라 판단했습니다
    - 2. 영향력이 적다.
    모델을 학습시켜 해당 숫자 토큰의 영향력을 판단해보니 앞도적으로 작아 사실상 거의 영향이 없었습니다.
    확인 방법은 모델을 학습시킨 이후, show_word_effect(['446'], n)과 같은 방식을 통해 확인해보았습니다.

- tokenize_text(text):  
text(이메일 내용)을 토큰으로 나누어주는 함수입니다. 토큰으로 나누는 기준은 여러가지 있겠지만, 여기서는 공백을 기준으로 문자를 분할하였습니다. 구체적으로는 정규 표현식을 사용하여 문자와 비문자의 경계인 \b를 이용해서 구분하였습니다.

- remove_stopwords(words):   
불용어에 해당되는 단어들은 words에서 제거하였습니다.

- generate_ngrams(words, n=2):
n-gram 방법에서 n을 설정하는 함수입니다. 기본값을 2로 설정해두었지만, 하이퍼파라미터 섹션에서 설정할 수 있습니다.   

In [410]:
words = preprocess_text('This this is IS 336 a 22 A 1 do doing done dont use usage play playing', **SELECT)
print(words)

['336', '22', '1', 'done', 'dont', 'use', 'usage', 'play', 'playing']


전처리의 결과를 간단히 확인할 수 있는 코드입니다.     
This, this의 경우, 포터 알고리즘을 거치면서 thi로 표현되었습니다. this라는 형태로 남았으면 좋았을것같지만, 하나의 어간으로 통일되어 기능상에는 큰 문제가 없을 것 같습니다.   

숫자의 경우, NUM을 바뀌었으나 불용어세 NUM을 추가하면서 토큰이 사라진걸 확인할 수 있습니다.

do, doing의 경우 불용어로 처리되어 사라졌습니다.   

playing은 포터 알고리즘에 의해 play로 변환되었습니다.

# Load Dataset

In [411]:
df = pd.read_csv('/content/spam_ham_dataset.csv')
X = df['text']
y = df['label_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

len(X_train), len(X_test)

(4136, 1035)

In [412]:
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


# 토큰 확인

In [413]:
word_counts = Counter()
for text in X_train: # text는 하나의 소문자화한 이메일 내용
    word_counts.update(preprocess_text(text, **SELECT))

# Show the 10 most common words
word_counts.most_common(10)

[('ect', 10731),
 ('subject', 6367),
 ('hou', 5630),
 ('enron', 5055),
 ('2000', 3468),
 ('com', 3126),
 ('please', 2564),
 ('gas', 2369),
 ('_', 2300),
 ('deal', 2218)]

In [414]:
len(word_counts)

45126

# Model(NaiveBayes Classifer)

In [415]:
class NaiveBayesClassifier:
    def __init__(self):
        self.word_probs = {}

    def fit(self, X, y, n):
        spam_word_counts = Counter()
        ham_word_counts = Counter()

        spam_count = 0
        ham_count = 0

        # Count words and emails
        for text, label in zip(X, y):
            tokens = preprocess_text(text, **SELECT)
            ngrams = generate_ngrams(tokens, n)

            if label == 1:  # spam
                spam_count += 1
                spam_word_counts.update(ngrams)
            else:  # ham
                ham_count += 1
                ham_word_counts.update(ngrams)

        # Calculate the total number of words in spam and ham emails
        total_spam_words = sum(spam_word_counts.values())
        total_ham_words = sum(ham_word_counts.values())

        # Calculate the total number of unique words
        total_unique_words = len(set(spam_word_counts.keys()).union(set(ham_word_counts.keys())))

        # Calculate the probabilities of spam and ham
        spam_prob = spam_count / (spam_count + ham_count)
        ham_prob = 1 - spam_prob

        # Calculate the word probabilities given spam and ham
        for word in set(spam_word_counts.keys()).union(set(ham_word_counts.keys())):
            spam_word_prob = (spam_word_counts[word] + 1) / (total_spam_words + total_unique_words)
            ham_word_prob = (ham_word_counts[word] + 1) / (total_ham_words + total_unique_words)
            self.word_probs[word] = (spam_word_prob, ham_word_prob)

        self.class_probs = (spam_prob, ham_prob)

    def predict(self, X, n):
        predictions = []
        for text in X:
            tokens = preprocess_text(text, **SELECT)
            ngrams = generate_ngrams(tokens, n)

            log_spam_prob = np.log(self.class_probs[0])
            log_ham_prob = np.log(self.class_probs[1])

            for word in ngrams:
                if word in self.word_probs:
                    log_spam_prob += np.log(self.word_probs[word][0])
                    log_ham_prob += np.log(self.word_probs[word][1])

            # Choose the class with higher log probability
            predictions.append(1 if log_spam_prob > log_ham_prob else 0)

        return np.array(predictions)

    def show_word_effect(self, X, n):
        predictions = []
        for text in X:
            tokens = preprocess_text(text, **SELECT)
            ngrams = generate_ngrams(tokens, n)
            print(ngrams)

            for word in ngrams:
                if word in self.word_probs:
                    log_spam_prob = np.log(self.class_probs[0])
                    log_ham_prob = np.log(self.class_probs[1])
                    log_spam_prob += np.log(self.word_probs[word][0])
                    log_ham_prob += np.log(self.word_probs[word][1])

                    print(f'token: {word}')
                    print(f'log_spam_prob: {log_spam_prob}, log_spam_prob: {log_spam_prob}')

                else:
                    print(f'token: {word}')
                    print(f'log_spam_prob: {0}, log_spam_prob: {0}')


nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(X_train, y_train, n=n)

나이브 베이즈를 기반으로 한 모델을 작성하였습니다.     

\<mothod fit\>
예측값을 내놓기 전, 학습 데이터를 기반으로 필요한 값들을 미리 구해놓는 작업을 수행합니다. 초기에 설정한 전처리 작업 이후 진행됩니다.

- self.word_probs:   
단어의 각 클래스(spam, ham)에 대한 우도를 저장하는 변수입니다. 딕셔너리로 작성되어 word값을 통해 각 클래스별로의 우도를 확인할 수 있습니다.   
ex) word_probs['boold'][0]: spam에 대한 'blood'의 우도   

- self.class_probs:  
각 클래스의 확률이 할당되는 튜플입니다.

- total_spam_words, total_ham_prob:   
각 단어별로 우도를 저장합니다. 이후 나이브 베이즈값을 계산할 떄 필요한 값들로 미리 구하여 저장해놓습니다.

- spam_word_counts, ham_word_counts:   
각각 Collections.Counter 객체를 할당하여 단어별 빈도수를 저장하였습니다. 단어별 클래스의 우도를 구하기 위하여 클래스별로 저장하였습니다.   

- spam_word_prob, ham_word_prob:    
이메일을 구성하는 토큰들을 기반으로 클래스별 우도를 저장하는 변수입니다.   
둘 중 큰 값을 고르게 되므로, 수식의 일부를 생략하였습니다.
(생략한 부분은 이메일 값에 대한 각 토큰들의 확률 곱으로 이루어져 있지만, 공통적으로 적용되는 부분이기에 생략해도 비교하는데 문제가 없습니다.)   
또한 새로운 단어에 대해 우도값이 0이 되는 것을 방지하지 위하여 '라플라스 스무딩'을 적용하였습니다. 방법은 우도에 1을 더하여 0이 되는것을 방지하고, 분모에는 모든 유니크한 단어의 갯수를 더하는것입니다. 이를 통해 합이 1이 되는 확률속성을 유지할 수 있습니다.   

\<mothod predict\>   
mothod fit에서 구한 값들을 이용해 예측값을 계산하는 함수입니다. 전처리 이후 진행됩니다.   

- log_spam_prob, log_ham_prob:   
각 클래스(spam, ham)의 확률을 할당받고, 각각 우도값을 곱한는 과정을 통해 값이 가장 큰 클래스를 결정하게 됩니다.   
해당 연산을 로그를 취하여 확률계산을 진행하였기데 덧셈으로 누적연산을 진행하였습니다. 이후 결과를 predictions 리스트에 저장하여 추후 label과의 비교를 통해 성능을 측정합니다.   

\<show_word_effect\>   
학습된 모델에서 각 토큰별로 미치는 영향을 출력하는 함수입니다. predict 메소드와 계산은 유사하지만, 한 토큰별로 결과를 출력합니다.



In [416]:
nb_classifier.show_word_effect(['this is the spam mail busniness 433  NUM augean'], n)

['spam mail busniness', 'mail busniness 433', 'busniness 433 num', '433 num augean']
token: spam mail busniness
log_spam_prob: 0, log_spam_prob: 0
token: mail busniness 433
log_spam_prob: 0, log_spam_prob: 0
token: busniness 433 num
log_spam_prob: 0, log_spam_prob: 0
token: 433 num augean
log_spam_prob: 0, log_spam_prob: 0


단어 혹은 문장을 집어넣어 각 토큰마다 클래스 결정에 얼만큼 결정했는지 확인하였습니다.



# Performance

In [417]:
def k_fold_cross_validation(X, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    f1_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize and train the Naive Bayes classifier
        nb_classifier = NaiveBayesClassifier()
        nb_classifier.fit(X_train, y_train, n)

        # Make predictions on the test set
        y_pred = nb_classifier.predict(X_test, n)

        # Calculate F1 score and accuracy
        f1 = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)

        f1_scores.append(f1)
        accuracy_scores.append(accuracy)

    return f1_scores, accuracy_scores

f1_scores, accuracy_scores = k_fold_cross_validation(X, y, k=5)
print("Mean F1 Score:", np.mean(f1_scores))
print("Mean Accuracy:", np.mean(accuracy_scores))

Mean F1 Score: 0.8165360693875814
Mean Accuracy: 0.9091088498304039


성능을 측정하는 방법으로 k fold cross validation을 사용하였습니다. k의 값은 하이퍼라마미터 섹션에서 CV를 통해서 조절할 수 있습니다.   

해당 예제에서는 k=5로 두고 평균값을 계산하였습니다.

In [418]:
# 데이터 프레임 생성

data = {
    'n=1': [0.96, 0.956, 0.959, 0.954, 0.95],
    'n=2': [0.92, 0.924, 0.938, 0.941, 0.925],
    'n=3': [0.881, 0.883, 0.816, 0.822, 0.839],
}

index = ['lower', 'stemming', 'loswer, stopwords','stemming, stopwords', 'stemming, stopwords, numreplace']
df1 = pd.DataFrame(data, index=index)


# 표 출력
print("n 별로의 결과값 테이블")
print(df1)

n 별로의 결과값 테이블
                                   n=1    n=2    n=3
lower                            0.960  0.920  0.881
stemming                         0.956  0.924  0.883
loswer, stopwords                0.959  0.938  0.938
stemming, stopwords              0.954  0.941  0.822
stemming, stopwords, numreplace  0.950  0.925  0.839


In [419]:
df1

,n=1,n=2,n=3
lower,0.960,0.920,0.881
stemming,0.956,0.924,0.883
"loswer, stopwords",0.959,0.938,0.938
"stemming, stopwords",0.954,0.941,0.822
"stemming, stopwords, numreplace",0.950,0.925,0.839


# 접근 방향

처음엔 의미없는 토근의 수가 많을수록 성능을 떨어트린다고 생각했었고, 따라서 전처리 과정에서 가능한 토큰수를 줄이는 방향으로 접근했습니다.   

처음 토근값을 살펴보았을 떄, 빈도수가 1인 단어들이 전체중에서 압도적으로 많았습니다. 그중에서 숫자들에 주목했었는데 아마 다시는 사용되지 않을것같다는 생각이 들었습니다. 숫자들을 처리하는 방법들을 생각해보다 제거하기보다는 대표문자로 변환하는 방법을 택했습니다. 이유는 의미없어보이는 숫자들이 스팸 이메일의 특징을 잘 나타낼지도 몰랐기 때문입니다.

토큰 값중에서 가장 빈도수가 높은 것은 the 였습니다. 불용어에 해당되는 단어가 압도적으로 많은 빈도수를 차지하는 특징을 보았고, 어디서든 사용될 만큼 범용적이라 없더라도 스팸 여부를 판단하는데는 지장이 없을거라 생각했습니다.

다음으로 어간을 추출하는 방법을 통해 의미가 같지만 형태만 달라 토근을 차지하는 가짓수를 줄여보았습니다.   



# 결과 분석

결과에서 가장 좋은 성능을 보였던 것은 1gram에서 소문자로 토큰을 추출한 경우였습니다.   

- n=1:   
    대체적으로 소문자화만 진행한 토큰이 성능이 가장 좋았습니다. 6000개 정도의 데이터셋이 작다고 생각해 토근을 줄이는것이 좋다고 판단했었는데, 텍스트 분류에서는 오히려 충분히 많은 데이터가 아니였나 싶습니다.

- n=2:   
    어간 추출이 유의미했으며, 기존 소문자만 진행했을때보다 0.004%의 성능 향상이 있었으며, 어간 추출을 추가했을 경우 기존에 비해 0.021% 상승했습니다.    

숫자를 추가한 경우, 전반적으로 기존보다 성능이 떨어졌는데 숫자에 예상보다 유의미한 정보들이 있었다고 판단됩니다.

